In [4]:
#####設定讀取資料的年份#####

year = 104    #110年或104年
rush_hours = ['晨峰', '昏峰']    # 同時處理晨峰和昏峰

################################

import pandas as pd
import numpy as np

def calculate_mape(actual, predicted):
    """計算 MAPE (Mean Absolute Percentage Error)"""
    if actual == 0:
        return 0.0  # 避免除以零
    return abs((actual - predicted) / actual) * 100

def calculate_overall_mape(actual_values, predicted_values):
    """計算整體 MAPE"""
    valid_pairs = [(a, p) for a, p in zip(actual_values, predicted_values) if a != 0]
    if not valid_pairs:
        return 0.0
    
    mape_values = [calculate_mape(a, p) for a, p in valid_pairs]
    return np.mean(mape_values)

def sort_group_names(group_names):
    """按照 SL1, SL2, ... 順序排序 group 名稱"""
    def extract_number(group_name):
        try:
            # 提取 SL 後面的數字
            if isinstance(group_name, str) and group_name.startswith('SL'):
                return int(group_name[2:])  # 取 SL 後面的數字部分
            else:
                return float('inf')  # 非 SL 格式的放在最後
        except:
            return float('inf')
    
    return sorted(group_names, key=extract_number)

# 初始化總合結果
all_results_comparison = []
overall_statistics = {}

# 對每個時段進行處理
for rush_hour in rush_hours:
    inputLocation = f"{year}_現況_{rush_hour}/"
    
    print(f'\n{"="*100}')
    print(f'處理 {year}年{rush_hour} 資料')
    print(f'{"="*100}')
    
    # 讀取 Excel 檔案，處理空值
    df = pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
    sheet1 = df["節點編號"]

    # 處理 sheet1 中的空值
    sheet1 = sheet1.fillna(0)  # 將空值填充為 0
    sheet1 = sheet1.replace('', 0)  # 將空字串替換為 0

    if rush_hour == '晨峰':
        real_flow = df["晨峰實際流量"]
    elif rush_hour == '昏峰':
        real_flow = df["昏峰實際流量"]

    # 處理 real_flow 中的空值
    real_flow = real_flow.fillna(0)
    real_flow = real_flow.replace('', 0)

    real_flow = real_flow.set_index("group")

    ue_results = pd.read_csv(inputLocation + f"{year}_現況_{rush_hour}_UE_results.dat", sep="\t")
    ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

    ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)

    # 建立 link_name 時處理可能的空值
    def safe_link_name(a, b):
        """安全地建立 link_name，處理空值"""
        try:
            if pd.isna(a) or pd.isna(b) or a == 0 or b == 0:
                return "0_0"  # 空值或 0 的情況
            return f"{int(a)}_{int(b)}"
        except (ValueError, TypeError):
            return "0_0"

    sheet1["link_name1"] = sheet1.apply(lambda row: safe_link_name(row['A1'], row['B1']), axis=1)
    sheet1["link_name2"] = sheet1.apply(lambda row: safe_link_name(row['A2'], row['B2']), axis=1)

    # 合併資料時處理空值
    sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], 
                          left_on="link_name1", right_on="link_name", how="left")
    sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], 
                          left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))

    # 處理合併後的空值
    sheet1['UE_flow_1'] = sheet1['UE_flow_1'].fillna(0)
    sheet1['UE_flow_2'] = sheet1['UE_flow_2'].fillna(0)

    # 保留需要的欄位
    sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

    # 處理 group 可能為空值的情況
    sheet1 = sheet1.dropna(subset=['group'])  # 移除 group 為空值的行

    group_names = sheet1['group'].unique()
    # 按照 SL1, SL2, ... 順序排序
    sorted_group_names = sort_group_names(group_names)

    groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in sorted_group_names}

    for index, row in sheet1.iterrows():
        group = row['group']
        # 確保流量值不是 NaN
        ue_flow_1 = row['UE_flow_1'] if not pd.isna(row['UE_flow_1']) else 0
        ue_flow_2 = row['UE_flow_2'] if not pd.isna(row['UE_flow_2']) else 0
        
        if group in groups:  # 確保 group 存在於排序後的字典中
            groups[group]['UE_flow_1'] += ue_flow_1
            groups[group]['UE_flow_2'] += ue_flow_2

    # 準備用於 MAPE 計算的列表
    ue_se_values = []
    actual_se_values = []
    cube_se_values = []
    ue_nw_values = []
    actual_nw_values = []
    cube_nw_values = []

    results_comparison = []

    print(f'——————————————————————詳細比較結果 ({rush_hour})——————————————————————')

    # 按照排序後的 group 順序處理
    # UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
    for group in sorted_group_names:
        flows = groups[group]
        try:
            # 檢查 real_flow 中是否有該 group 的資料
            if group not in real_flow.index:
                print(f"【{group}】 資料不完整，跳過計算")
                continue
                
            # 安全地取得實際流量數據
            name = real_flow.loc[group, 'name'] if 'name' in real_flow.columns else f"Group {group}"
            south_east = real_flow.loc[group, '南/東'] if '南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, '南/東']) else 1
            north_west = real_flow.loc[group, '北/西'] if '北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, '北/西']) else 1
            
            # 計算差異百分比，避免除以零
            model_diff_se = (flows['UE_flow_1'] - south_east) / south_east * 100 if south_east != 0 else 0
            model_diff_nw = (flows['UE_flow_2'] - north_west) / north_west * 100 if north_west != 0 else 0
            
            # CUBE 差異計算（如果有 CUBE 資料）
            cube_se = real_flow.loc[group, 'CUBE南/東'] if 'CUBE南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE南/東']) else south_east
            cube_nw = real_flow.loc[group, 'CUBE北/西'] if 'CUBE北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE北/西']) else north_west
            
            cube_diff_se = (cube_se - south_east) / south_east * 100 if south_east != 0 else 0
            cube_diff_nw = (cube_nw - north_west) / north_west * 100 if north_west != 0 else 0
            
            # 計算 MAPE
            ue_mape_se = calculate_mape(south_east, flows['UE_flow_1'])
            ue_mape_nw = calculate_mape(north_west, flows['UE_flow_2'])
            cube_mape_se = calculate_mape(south_east, cube_se)
            cube_mape_nw = calculate_mape(north_west, cube_nw)
            
            # 收集數據用於整體 MAPE 計算
            if south_east > 0:  # 只計算非零實際值
                ue_se_values.append(flows['UE_flow_1'])
                actual_se_values.append(south_east)
                cube_se_values.append(cube_se)
                
            if north_west > 0:  # 只計算非零實際值
                ue_nw_values.append(flows['UE_flow_2'])
                actual_nw_values.append(north_west)
                cube_nw_values.append(cube_nw)
            
            # 儲存比較結果（加上時段標識）
            results_comparison.append({
                'rush_hour': rush_hour,
                'group': group,
                'name': name,
                'actual_se': south_east,
                'ue_se': flows['UE_flow_1'],
                'cube_se': cube_se,
                'actual_nw': north_west,
                'ue_nw': flows['UE_flow_2'],
                'cube_nw': cube_nw,
                'ue_mape_se': ue_mape_se,
                'cube_mape_se': cube_mape_se,
                'ue_mape_nw': ue_mape_nw,
                'cube_mape_nw': cube_mape_nw
            })
            
            print(f"""【{group}】 {name}
        往南/東流量:
            實際流量 = {south_east:,.0f}
            UE 模型 = {round(flows['UE_flow_1']):,} (差異: {model_diff_se:+.2f}%, MAPE: {ue_mape_se:.2f}%)
            CUBE 模型 = {cube_se:,.0f} (差異: {cube_diff_se:+.2f}%, MAPE: {cube_mape_se:.2f}%)
        往北/西流量:
            實際流量 = {north_west:,.0f}
            UE 模型 = {round(flows['UE_flow_2']):,} (差異: {model_diff_nw:+.2f}%, MAPE: {ue_mape_nw:.2f}%)
            CUBE 模型 = {cube_nw:,.0f} (差異: {cube_diff_nw:+.2f}%, MAPE: {cube_mape_nw:.2f}%)
        """)
            
        except Exception as e:
            print(f"【{group}】 處理時發生錯誤: {e}")
            continue

    # 計算該時段的整體 MAPE
    print(f'\n{"="*80}')
    print(f'整體模型表現評估 ({year}年{rush_hour})')
    print(f'{"="*80}')

    # 往南/東方向整體 MAPE
    ue_overall_mape_se = calculate_overall_mape(actual_se_values, ue_se_values)
    cube_overall_mape_se = calculate_overall_mape(actual_se_values, cube_se_values)

    # 往北/西方向整體 MAPE
    ue_overall_mape_nw = calculate_overall_mape(actual_nw_values, ue_nw_values)
    cube_overall_mape_nw = calculate_overall_mape(actual_nw_values, cube_nw_values)

    # 全方向整體 MAPE
    all_actual = actual_se_values + actual_nw_values
    all_ue = ue_se_values + ue_nw_values
    all_cube = cube_se_values + cube_nw_values

    ue_overall_mape = calculate_overall_mape(all_actual, all_ue)
    cube_overall_mape = calculate_overall_mape(all_actual, all_cube)

    print(f"""
    往南/東方向 MAPE:
        UE 模型: {ue_overall_mape_se:.2f}%
        CUBE 模型: {cube_overall_mape_se:.2f}%
        差異: {ue_overall_mape_se - cube_overall_mape_se:+.2f}% (負值表示 UE 表現較好)

    往北/西方向 MAPE:
        UE 模型: {ue_overall_mape_nw:.2f}%
        CUBE 模型: {cube_overall_mape_nw:.2f}%
        差異: {ue_overall_mape_nw - cube_overall_mape_nw:+.2f}% (負值表示 UE 表現較好)

    全方向整體 MAPE:
        UE 模型: {ue_overall_mape:.2f}%
        CUBE 模型: {cube_overall_mape:.2f}%
        差異: {ue_overall_mape - cube_overall_mape:+.2f}% (負值表示 UE 表現較好)

    統計摘要:
        評估路段數量: {len(results_comparison)}
        有效南/東流量比較: {len(actual_se_values)}
        有效北/西流量比較: {len(actual_nw_values)}
    """)
    
    # 儲存該時段的統計結果
    overall_statistics[rush_hour] = {
        'ue_overall_mape_se': ue_overall_mape_se,
        'cube_overall_mape_se': cube_overall_mape_se,
        'ue_overall_mape_nw': ue_overall_mape_nw,
        'cube_overall_mape_nw': cube_overall_mape_nw,
        'ue_overall_mape': ue_overall_mape,
        'cube_overall_mape': cube_overall_mape,
        'sample_count': len(results_comparison),
        'se_count': len(actual_se_values),
        'nw_count': len(actual_nw_values)
    }
    
    # 將該時段的結果加入總合
    all_results_comparison.extend(results_comparison)

# 創建包含所有時段的詳細比較表格
comparison_df = pd.DataFrame(all_results_comparison)

print(f'\n{"="*100}')
print(f'合併所有時段的詳細比較表格 ({year}年晨昏峰)')
print(f'{"="*100}')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(comparison_df.round(2))

# 找出表現最好和最差的路段（合併兩時段）
if len(comparison_df) > 0:
    print(f'\n{"="*100}')
    print('模型表現分析 (晨昏峰合併)')
    print(f'{"="*100}')
    
    # UE 模型表現最好的路段
    best_ue_se = comparison_df.loc[comparison_df['ue_mape_se'].idxmin()]
    worst_ue_se = comparison_df.loc[comparison_df['ue_mape_se'].idxmax()]
    
    print(f"""
UE 模型往南/東方向表現:
    最佳路段: 【{best_ue_se['group']}】{best_ue_se['name']} ({best_ue_se['rush_hour']}) (MAPE: {best_ue_se['ue_mape_se']:.2f}%)
    最差路段: 【{worst_ue_se['group']}】{worst_ue_se['name']} ({worst_ue_se['rush_hour']}) (MAPE: {worst_ue_se['ue_mape_se']:.2f}%)
    
CUBE vs UE 比較 (所有樣本):
    UE 表現較好的路段數量: {sum(comparison_df['ue_mape_se'] < comparison_df['cube_mape_se'])}
    CUBE 表現較好的路段數量: {sum(comparison_df['cube_mape_se'] < comparison_df['ue_mape_se'])}
    表現相同的路段數量: {sum(comparison_df['ue_mape_se'] == comparison_df['cube_mape_se'])}
    
各時段統計摘要:
    """)
    
    for rush_hour in rush_hours:
        stats = overall_statistics[rush_hour]
        print(f"""  {rush_hour}:
        評估路段數量: {stats['sample_count']}
        有效南/東流量比較: {stats['se_count']}
        有效北/西流量比較: {stats['nw_count']}
        UE 整體MAPE: {stats['ue_overall_mape']:.2f}%
        CUBE 整體MAPE: {stats['cube_overall_mape']:.2f}%""")
    
    print(f"""
總計樣本數: {len(comparison_df)} (期望: 52 = 13條屏柵線 × 2時段 × 2方向)
    """)

# 顯示最終合併的表格摘要
print(f'\n{"="*100}')
print('最終資料摘要 (晨昏峰合併)')
print(f'{"="*100}')

# 按時段和group統計
summary_by_period = comparison_df.groupby(['rush_hour', 'group']).size().reset_index(name='count')
print("各時段各Group的資料筆數:")
print(summary_by_period.pivot(index='group', columns='rush_hour', values='count').fillna(0))

print(f"""
總計資料統計:
    總樣本數: {len(comparison_df)}
    晨峰樣本數: {len(comparison_df[comparison_df['rush_hour'] == '晨峰'])}
    昏峰樣本數: {len(comparison_df[comparison_df['rush_hour'] == '昏峰'])}
    期望樣本數: 52 (13條屏柵線 × 2時段 × 2方向)
    實際達成率: {len(comparison_df)/52*100:.1f}%
""")


處理 104年晨峰 資料
——————————————————————詳細比較結果 (晨峰)——————————————————————
【SL1】 基隆河
        往南/東流量:
            實際流量 = 42,683
            UE 模型 = 42,656 (差異: -0.06%, MAPE: 0.06%)
            CUBE 模型 = 46,390 (差異: +8.69%, MAPE: 8.69%)
        往北/西流量:
            實際流量 = 42,362
            UE 模型 = 45,159 (差異: +6.60%, MAPE: 6.60%)
            CUBE 模型 = 42,262 (差異: -0.24%, MAPE: 0.24%)
        
【SL2】 台鐵(市民大道)
        往南/東流量:
            實際流量 = 29,823
            UE 模型 = 32,444 (差異: +8.79%, MAPE: 8.79%)
            CUBE 模型 = 34,108 (差異: +14.37%, MAPE: 14.37%)
        往北/西流量:
            實際流量 = 38,481
            UE 模型 = 34,475 (差異: -10.41%, MAPE: 10.41%)
            CUBE 模型 = 37,853 (差異: -1.63%, MAPE: 1.63%)
        
【SL3】 國1-市中心
        往南/東流量:
            實際流量 = 6,162
            UE 模型 = 5,718 (差異: -7.21%, MAPE: 7.21%)
            CUBE 模型 = 5,069 (差異: -17.74%, MAPE: 17.74%)
        往北/西流量:
            實際流量 = 5,399
            UE 模型 = 5,246 (差異: -2.83%, MAPE: 2.83%)
            CUBE 模型 = 4,679 

In [5]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table

,group,link,UE_flow_1,UE_flow_2
4,SL1,中山橋,1089.852627,1131.453663
13,SL1,南湖大橋,2044.172866,2022.037352
5,SL1,大直橋,2728.002893,2489.053244
12,SL1,成功橋,1872.031151,1537.524656
11,SL1,成美橋,2088.356460,1580.133154
2,SL1,承德橋,1266.464040,2268.671649
3,SL1,承德橋(機車專用道),991.214465,1134.419220
7,SL1,民權大橋,1642.141009,1471.766574
8,SL1,民權大橋(機車專用道),1291.793733,878.407189
0,SL1,洲美快速道路,3577.868389,4753.351736


In [6]:
# 配對樣本 t 檢定分析
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print(f'\n{"="*80}')
print('配對樣本 t 檢定分析')
print(f'{"="*80}')

# 收集所有MAPE數據進行配對樣本t檢定
# 包括南/東和北/西方向的所有有效數據
ue_mape_all = []
cube_mape_all = []

# 從comparison_df收集所有有效的MAPE數據
for _, row in comparison_df.iterrows():
    # 南/東方向數據
    if row['actual_se'] > 0:  # 只包含有效的實際流量
        ue_mape_all.append(row['ue_mape_se'])
        cube_mape_all.append(row['cube_mape_se'])
    
    # 北/西方向數據
    if row['actual_nw'] > 0:  # 只包含有效的實際流量
        ue_mape_all.append(row['ue_mape_nw'])
        cube_mape_all.append(row['cube_mape_nw'])

# 轉換為numpy陣列
ue_mape_array = np.array(ue_mape_all)
cube_mape_array = np.array(cube_mape_all)

# 計算基本統計
ue_mape_mean = np.mean(ue_mape_array)
cube_mape_mean = np.mean(cube_mape_array)
ue_mape_std = np.std(ue_mape_array, ddof=1)
cube_mape_std = np.std(cube_mape_array, ddof=1)

# 進行配對樣本t檢定
# H0: UE模型和CUBE模型的MAPE沒有顯著差異
# H1: UE模型和CUBE模型的MAPE有顯著差異
t_statistic, p_value = stats.ttest_rel(ue_mape_array, cube_mape_array)

# 計算效應大小 (Cohen's d for paired samples)
diff_array = ue_mape_array - cube_mape_array
mean_diff = np.mean(diff_array)
std_diff = np.std(diff_array, ddof=1)
cohens_d = mean_diff / std_diff

print(f"""
樣本統計摘要:
    總樣本數: {len(ue_mape_array)}
    期望樣本數: 52 (13條屏柵線 × 晨昏峰 × 雙向)
    
UE 模型 MAPE:
    平均值: {ue_mape_mean:.4f}%
    標準差: {ue_mape_std:.4f}%
    
CUBE 模型 MAPE:
    平均值: {cube_mape_mean:.4f}%
    標準差: {cube_mape_std:.4f}%

配對差異 (UE - CUBE):
    平均差異: {mean_diff:.4f}%
    標準差: {std_diff:.4f}%

配對樣本 t 檢定結果:
    t 統計量: {t_statistic:.6f}
    p-value: {p_value:.6f}
    自由度: {len(ue_mape_array) - 1}
    效應大小 (Cohen's d): {cohens_d:.4f}

統計結論:
""")

# 判斷統計顯著性
alpha = 0.05
if p_value < alpha:
    significance = "統計上顯著"
    if mean_diff < 0:
        conclusion = "UE模型的MAPE顯著低於CUBE模型，表現較佳"
    else:
        conclusion = "UE模型的MAPE顯著高於CUBE模型，表現較差"
else:
    significance = "統計上不顯著"
    conclusion = "兩個模型的MAPE沒有顯著差異"

print(f"    在α = {alpha}的顯著水準下，差異{significance}")
print(f"    結論: {conclusion}")

# 效應大小解釋
if abs(cohens_d) < 0.2:
    effect_size = "小"
elif abs(cohens_d) < 0.5:
    effect_size = "中等"
elif abs(cohens_d) < 0.8:
    effect_size = "大"
else:
    effect_size = "非常大"

print(f"    效應大小: {effect_size} (|Cohen's d| = {abs(cohens_d):.4f})")

# 信賴區間計算
from scipy.stats import t as t_dist
confidence_level = 0.95
df = len(diff_array) - 1
t_critical = t_dist.ppf((1 + confidence_level) / 2, df)
margin_error = t_critical * (std_diff / np.sqrt(len(diff_array)))
ci_lower = mean_diff - margin_error
ci_upper = mean_diff + margin_error

print(f"""
{confidence_level*100}% 信賴區間:
    平均差異的信賴區間: [{ci_lower:.4f}%, {ci_upper:.4f}%]
""")

# 詳細的配對數據表格
print(f'\n{"="*80}')
print('詳細配對數據 (前10筆樣本)')
print(f'{"="*80}')

paired_data = pd.DataFrame({
    'UE_MAPE': ue_mape_array,
    'CUBE_MAPE': cube_mape_array,
    'Difference': diff_array
})

print(paired_data.head(10).round(4))
print(f"... (共 {len(paired_data)} 筆數據)")

# 額外統計資訊
print(f'\n{"="*80}')
print('額外統計資訊')
print(f'{"="*80}')
print(f"UE模型表現較好的樣本數: {sum(diff_array < 0)} ({sum(diff_array < 0)/len(diff_array)*100:.1f}%)")
print(f"CUBE模型表現較好的樣本數: {sum(diff_array > 0)} ({sum(diff_array > 0)/len(diff_array)*100:.1f}%)")
print(f"兩模型表現相同的樣本數: {sum(diff_array == 0)} ({sum(diff_array == 0)/len(diff_array)*100:.1f}%)")


配對樣本 t 檢定分析

樣本統計摘要:
    總樣本數: 52
    期望樣本數: 52 (13條屏柵線 × 晨昏峰 × 雙向)
    
UE 模型 MAPE:
    平均值: 8.7601%
    標準差: 7.1311%
    
CUBE 模型 MAPE:
    平均值: 8.8584%
    標準差: 5.6481%

配對差異 (UE - CUBE):
    平均差異: -0.0983%
    標準差: 8.7407%

配對樣本 t 檢定結果:
    t 統計量: -0.081072
    p-value: 0.935702
    自由度: 51
    效應大小 (Cohen's d): -0.0112

統計結論:

    在α = 0.05的顯著水準下，差異統計上不顯著
    結論: 兩個模型的MAPE沒有顯著差異
    效應大小: 小 (|Cohen's d| = 0.0112)

95.0% 信賴區間:
    平均差異的信賴區間: [-2.5317%, 2.3352%]


詳細配對數據 (前10筆樣本)
   UE_MAPE  CUBE_MAPE  Difference
0   0.0647     8.6850     -8.6204
1   6.6038     0.2355      6.3684
2   8.7883    14.3696     -5.5813
3  10.4118     1.6321      8.7797
4   7.2114    17.7374    -10.5260
5   2.8252    13.3328    -10.5076
6  17.4355     4.6853     12.7502
7  18.7753     8.0348     10.7405
8   1.2324    17.1691    -15.9366
9   6.6996    19.6071    -12.9075
... (共 52 筆數據)

額外統計資訊
UE模型表現較好的樣本數: 30 (57.7%)
CUBE模型表現較好的樣本數: 22 (42.3%)
兩模型表現相同的樣本數: 0 (0.0%)
